# 使用resnet完成头部检测任务

导入数据类

In [17]:
from PIL import Image
from torch.utils import data

class Mydataset(data.Dataset):
    def __init__(self, img_paths, labels, transform):
        self.imgs = img_paths
        self.labels = labels
        self.transforms = transform

    def __getitem__(self, item):
        img = self.imgs[item]
        label = self.labels[item]
        pil_img = Image.open(img)
        pil_img = pil_img.convert("RGB")
        data = self.transforms(pil_img)
        return data, label

    def __len__(self):
        return len(self.imgs)

导入resnet模型

In [18]:
import torch
import torch.nn as nn

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=3):
        self.inplanes = 64
        super(ResNet, self).__init__()
        # 调整卷积层以适应 128x128 输入
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        # 移除最大池化层以保留更多空间信息
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet50(pretrained=False, pretrained_path=None, **kwargs):
    """Constructs a modified ResNet-50 model for 128x128 input.
    
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet (not applicable for modified model).
        pretrained_path (str): Path to the .pth file with pretrained weights.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained_path:
        model.load_state_dict(torch.load(pretrained_path))
    return model


导入数据

In [19]:
import glob
import numpy as np
from PIL import Image
import os
from torch.utils import data
from torchvision import transforms
def read_last_three_columns(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        last_three_columns = [list(map(float, line.strip().split(' ')[-3:])) for line in lines]
    return last_three_columns

imagePaths = glob.glob("Data/genki4k/files/*.jpg")
labelPaths = read_last_three_columns("Data/genki4k/labels.txt")

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])
index = np.random.permutation(len(imagePaths))
all_imgs_path = np.array(imagePaths)[index]
all_labels = np.array(labelPaths)[index]

# 保存
save_dir = 'save'
os.makedirs(save_dir, exist_ok=True)

# 定义训练集、验证集和测试集的大小
train_size = int(len(all_imgs_path) * 0.7)
val_size = int(len(all_imgs_path) * 0.15)

# 划分训练集、验证集和测试集
train_imgs = all_imgs_path[:train_size]
train_labels = all_labels[:train_size]
val_imgs = all_imgs_path[train_size:train_size+val_size]
val_labels = all_labels[train_size:train_size+val_size]
test_imgs = all_imgs_path[train_size+val_size:]
test_labels = all_labels[train_size+val_size:]

# 创建数据集
train_ds = Mydataset(train_imgs, train_labels, transform)
val_ds = Mydataset(val_imgs, val_labels, transform)
test_ds = Mydataset(test_imgs, test_labels, transform)

# 创建数据加载器
train_dl = data.DataLoader(train_ds, batch_size=4, shuffle=True)
val_dl = data.DataLoader(val_ds, batch_size=4, shuffle=False)
test_dl = data.DataLoader(test_ds, batch_size=4, shuffle=False)


定义训练函数

In [20]:
from tqdm import tqdm

def fit(epoch, model, loss_fn, optim, train_dl, test_dl):
    model.train()
    train_losses = []
    train_loader = tqdm(train_dl, desc=f'Epoch {epoch+1}/{epochs}', unit='batch')

    for x, y in train_loader:
        x = x.float()
        y = y.float()
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        optim.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optim.step()

        train_losses.append(loss.item())

        # 更新进度条
        train_loader.set_postfix(loss=np.mean(train_losses))

    train_loss = np.mean(train_losses)

    model.eval()
    test_losses = []
    with torch.no_grad():
        for x, y in test_dl:
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            test_losses.append(loss.item())

    test_loss = np.mean(test_losses)
    print(f'Epoch: {epoch}, Train Loss: {train_loss:.3f}, Test Loss: {test_loss:.3f}')
    return test_loss


进行训练

In [24]:
model = resnet50(True,"save/Task2-2/model_last_epoch.pth")  # 确保这是适合任务的 ResNet 变体
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)  # 移动模型到cuda

loss_fn = nn.MSELoss()  # 使用 MSE 损失函数
optim = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 200

best_loss = float('inf')
save_dir = 'save/Task2-3'
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    train_loss = fit(epoch, model, loss_fn, optim, train_dl, val_dl)  # fit 函数需要返回训练损失
    if train_loss < best_loss:
        best_loss = train_loss
        best_model_path = os.path.join(save_dir, 'model_best_loss.pth')
        torch.save(model.state_dict(), best_model_path)

    # 保存最后一个 epoch 的模型
    last_model_path = os.path.join(save_dir, 'model_last_epoch.pth')
    torch.save(model.state_dict(), last_model_path)

print(f"Best model saved to {best_model_path}")
print(f"Last epoch model saved to {last_model_path}")
print("Finish Training!")


Epoch 1/200: 100%|██████████| 700/700 [00:29<00:00, 23.66batch/s, loss=0.00206]


Epoch: 0, Train Loss: 0.002, Test Loss: 0.010


Epoch 2/200: 100%|██████████| 700/700 [00:29<00:00, 24.07batch/s, loss=0.00189]


Epoch: 1, Train Loss: 0.002, Test Loss: 0.014


Epoch 3/200: 100%|██████████| 700/700 [00:29<00:00, 23.88batch/s, loss=0.00193]


...

Epoch: 197, Train Loss: 0.000, Test Loss: 0.008


Epoch 199/200: 100%|██████████| 700/700 [00:28<00:00, 24.70batch/s, loss=0.000211]


Epoch: 198, Train Loss: 0.000, Test Loss: 0.008


Epoch 200/200: 100%|██████████| 700/700 [00:29<00:00, 23.90batch/s, loss=0.000205]


Epoch: 199, Train Loss: 0.000, Test Loss: 0.008
Best model saved to save/Task2-3/model_best_loss.pth
Last epoch model saved to save/Task2-3/model_last_epoch.pth
Finish Training!


测试